In [1]:
#pertama kita masukkan library yang akan digunakan
library(C50)
library(openxlsx)
library(reshape2)

Warning message:
"package 'C50' was built under R version 3.6.3"
Warning message:
"package 'openxlsx' was built under R version 3.6.3"
Warning message:
"package 'reshape2' was built under R version 3.6.3"


In [2]:
#lalu kita import data, kali ini saya meminjam dataset dari DQLab Academy
dataCredit <- read.xlsx(xlsxFile = "https://academy.dqlab.id/dataset/credit_scoring_dqlab.xlsx")
head(dataCredit)

,kode_kontrak,pendapatan_setahun_juta,kpr_aktif,durasi_pinjaman_bulan,jumlah_tanggungan,rata_rata_overdue,risk_rating
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>
1,AGR-000001,295,YA,48,5,61 - 90 days,4
2,AGR-000011,271,YA,36,5,61 - 90 days,4
3,AGR-000030,159,TIDAK,12,0,0 - 30 days,1
4,AGR-000043,210,YA,12,3,46 - 60 days,3
5,AGR-000049,165,TIDAK,36,0,31 - 45 days,2
6,AGR-000063,220,TIDAK,24,5,0 - 30 days,1


In [3]:
#Mempersiapkan class dan input variables 
dataCredit$risk_rating <- as.factor(dataCredit$risk_rating)
input_kolom <- c("durasi_pinjaman_bulan","jumlah_tanggungan","kpr_aktif","pendapatan_setahun_juta")
datafeed <- dataCredit[input_kolom]
str(datafeed)

'data.frame':	900 obs. of  4 variables:
 $ durasi_pinjaman_bulan  : num  48 36 12 12 36 24 36 48 48 36 ...
 $ jumlah_tanggungan      : num  5 5 0 3 0 5 3 3 5 6 ...
 $ kpr_aktif              : chr  "YA" "YA" "TIDAK" "YA" ...
 $ pendapatan_setahun_juta: num  295 271 159 210 165 220 70 88 163 100 ...


In [4]:
#Mempersiapkan training dan testing set
set.seed(100) #untuk menyeragamkan hasil random antar tiap komputer
xtrain_value <- sample(900, 800)

In [5]:
#Membuat training set dan testing set
xtrain <- datafeed[xtrain_value,]
class_xtrain <- dataCredit[xtrain_value,]$risk_rating
xtest <- datafeed[-xtrain_value,]

In [6]:
#menghasilkan model train dan menampilkan summary
model_risk_rating <- C5.0(xtrain, class_xtrain, control = C5.0Control(label="Risk Rating"))
summary(model_risk_rating)


Call:
C5.0.default(x = xtrain, y = class_xtrain, control = C5.0Control(label =
 "Risk Rating"))


C5.0 [Release 2.07 GPL Edition]  	Thu Jul 02 17:25:52 2020
-------------------------------

Class specified by attribute `Risk Rating'

Read 800 cases (5 attributes) from undefined.data

Decision tree:

kpr_aktif = YA:
:...jumlah_tanggungan <= 4: 3 (255/4)
:   jumlah_tanggungan > 4:
:   :...pendapatan_setahun_juta > 248: 4 (52)
:       pendapatan_setahun_juta <= 248:
:       :...durasi_pinjaman_bulan <= 24: 4 (71/17)
:           durasi_pinjaman_bulan > 24: 5 (72/4)
kpr_aktif = TIDAK:
:...pendapatan_setahun_juta <= 95: 2 (36/7)
    pendapatan_setahun_juta > 95:
    :...durasi_pinjaman_bulan > 36: 2 (37/4)
        durasi_pinjaman_bulan <= 36:
        :...pendapatan_setahun_juta > 201: 1 (50)
            pendapatan_setahun_juta <= 201:
            :...durasi_pinjaman_bulan > 24: 2 (73/33)
                durasi_pinjaman_bulan <= 24:
                :...jumlah_tanggungan > 2:
                

In [7]:
#model sudah belajar, mari mulai kita predict
hasil_predik <- predict(model_risk_rating, xtest)
summary(hasil_predik)

1  2  3  4  5 
20 17 38  9 16

In [8]:
#masukkan hasil prediksi ke dalam dataframe xtest
xtest$risk_rating <- dataCredit[-xtrain_value,]$risk_rating
xtest$hasil_prediksi <- hasil_predik

In [9]:
#melihat perbandingan data dengan confusion matrix
dcast(hasil_prediksi ~ risk_rating, data=xtest)

Using hasil_prediksi as value column: use value.var to override.

Aggregation function missing: defaulting to length



hasil_prediksi,1,2,3,4,5
<fct>,<int>,<int>,<int>,<int>,<int>
1,20,0,0,0,0
2,4,13,0,0,0
3,0,0,38,0,0
4,0,0,0,9,0
5,0,0,0,0,16


In [10]:
#menghitung berapa banyak prediksi data yang benar
nrow(xtest[xtest$risk_rating == xtest$hasil_prediksi,])

[1] 96

### Terlihat bahwa jumlah data yang benar diprediksi adalah 96 dari 100 data test, maka akurasi model machine learning kita adalah sebesar 96%